In [6]:
import fastai
import torch

from fastai.tabular import *

In [7]:
# Check if we're using the GPU.
torch.cuda.get_device_name(0)

'GeForce GTX 1080'

In [8]:
pd

<module 'pandas' from '/home/lacker/miniconda3/envs/snecko/lib/python3.7/site-packages/pandas/__init__.py'>